In [1]:
import pandas as pd
import numpy as np
import datetime
from data import LoadSales
import logging as log
from models import *

In [ ]:
data = LoadSales('teradata_sales')

In [ ]:
df_i = data.transform(freq ="month",sku = 69239730)

In [ ]:
data = df_i.sales
# data split
n_test = 14
# model configs
cfg_list = exp_smoothing_configs(seasonal=[12])

In [ ]:
scores = grid_search(data[:,0], cfg_list, n_test)
print('done')
# list top 3 configs
for cfg, error in scores[:3]:
    print(cfg, error)


In [5]:
# model configs
#cfg_list = exp_smoothing_configs(seasonal=[0,6,12])

In [8]:

data = series.values
# data split
n_test = 12
# model configs
cfg_list = exp_smoothing_configs(seasonal=[0,6,12])
# grid search
scores = grid_search(data[:,0], cfg_list, n_test)
print('done')
# list top 3 configs
for cfg, error in scores[:3]:
    print(cfg, error)


 > Model[['add', True, 'add', 6, False, False]] 3226.384
 > Model[['add', True, 'add', 6, False, True]] 3240.433
 > Model[['add', True, 'add', 6, True, False]] 2800.712
 > Model[['add', True, 'add', 6, True, True]] 2802.612


KeyboardInterrupt: 

In [87]:
# grid search
scores = grid_search(np.array(data), cfg_list, n_test)

 > Model[['add', True, 'add', 12, False, False]] 301.025
 > Model[['add', True, 'add', 12, False, True]] 293.210
 > Model[['add', True, 'mul', 12, False, True]] 300.670
 > Model[['add', True, 'mul', 12, False, False]] 307.746
 > Model[['add', True, None, 12, False, True]] 190.812
 > Model[['add', True, None, 12, False, False]] 193.921
 > Model[['add', False, 'add', 12, False, True]] 289.850
 > Model[['add', False, 'add', 12, False, False]] 297.324
 > Model[['add', False, None, 12, False, True]] 190.865
 > Model[['add', False, None, 12, False, False]] 189.543
 > Model[['add', False, 'mul', 12, False, True]] 299.393
 > Model[['add', False, 'mul', 12, False, False]] 299.680
 > Model[['mul', True, 'add', 12, False, True]] 407.592
 > Model[['mul', True, 'add', 12, False, False]] 380.924
 > Model[['mul', True, None, 12, False, True]] 190.812
 > Model[['mul', False, 'add', 12, True, False]] 271.648
 > Model[['mul', True, None, 12, False, False]] 193.950
 > Model[['mul', False, 'add', 12, Fals

In [103]:
print('done')
# list top 3 configs
for cfg, error in scores[:3]:
    print(cfg, error)

done
['mul', False, None, 12, False, False] 118.35147763511883
['mul', False, None, 12, False, True] 151.77278262656867
['add', False, None, 12, False, False] 189.5431548235939


In [108]:
# one-step Holt Winter's Exponential Smoothing forecast
def exp_smoothing_forecast(history, config):
	t,d,s,p,b,r = config
	# define model model
	history = array(history)
	model = ExponentialSmoothing(history, trend=t, damped=d, seasonal=s, seasonal_periods=p)
	# fit model
	model_fit = model.fit(optimized=True, use_boxcox=b, remove_bias=r)
	# make one step forecast
	yhat = 
	return yhat[0]

In [111]:
exp_smoothing_forecast(df_i.sales,['mul', False, None, 12, False, False])

304.3233067816989

In [112]:
t,d,s,p,b,r = ['mul', False, None, 12, False, False]

In [116]:
history = array(df_i.sales)

In [162]:
model = ExponentialSmoothing(history, trend=t, damped=d, seasonal=s, seasonal_periods=p)


In [168]:
model_fit = model.fit(optimized=True, use_boxcox=b, remove_bias=r)


In [169]:
model_fit.params

{'smoothing_level': 0.011473080314533951,
 'smoothing_slope': 0.0,
 'smoothing_seasonal': nan,
 'damping_slope': nan,
 'initial_level': 429.00017257241626,
 'initial_slope': 0.9912897149982722,
 'initial_seasons': array([], dtype=float64),
 'use_boxcox': False,
 'lamda': None,
 'remove_bias': False}

In [171]:
predictions = model_fit.predict(start=len(data), end = len(data)+ 10)


In [175]:
model_fit.predict(start=len(data), end = len(data)+ 10)

array([304.32330678, 301.67256405, 299.04491004, 296.44014364,
       293.85806551, 291.298478  , 288.76118524, 286.24599302,
       283.75270884, 281.28114188, 278.83110296])

In [177]:
model_fit.predict(start=len(data), end = len(data)+ 10)


array([304.32330678, 301.67256405, 299.04491004, 296.44014364,
       293.85806551, 291.298478  , 288.76118524, 286.24599302,
       283.75270884, 281.28114188, 278.83110296])